<a href="https://colab.research.google.com/github/Illumaria/iti-funds-nav-parser/blob/master/iti_ruse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [0]:
url = 'https://www.londonstockexchange.com/stock/RUSE/iti-funds-ucits-etf-sicav/analysis?page=1' # initial link (unspecified page)

In [0]:
iti_resp = requests.get(url)

In [0]:
bs = BeautifulSoup(iti_resp.text)

In [10]:
# Get the number of pages:
pages = bs.find('a', class_="page-number page-last")
pages
#num_pages = int(pages.split()[3])

<a _ngcontent-sc27="" class="page-number page-last" href="/stock/RUSE/iti-funds-ucits-etf-sicav/analysis?page=6" rel="nofollow">Last page</a>

In [0]:
# Find all the partial links on the first page:
scripts_ruse = bs.find_all('a', text = re.compile('ETF SICAV: Net Asset Value'))
scripts_rusb = bs.find_all('a', text = re.compile('EUROBOND: Net Asset Value'))

In [0]:
# Get full links:
urls = []
urls.extend(['https://www.londonstockexchange.com' + scripts[i].get('href').split("'")[1] for i in range(len(scripts))])

In [0]:
# Do the same for other pages:
for i in range(2,num_pages+1):
  resp = requests.get('{}{}{}'.format(url, "&page=",i))
  bs = BeautifulSoup(resp.text)
  scripts = bs.find_all('a', text = re.compile('Net Asset Value'))
  urls.extend(['https://www.londonstockexchange.com' + scripts[i].get('href').split("'")[1] for i in range(len(scripts))])

In [0]:
# Get the number of urls to determine the number of table rows:
nrows = len(urls)
# Create an empty pandas dataframe with 0 to (nrows-1) rows and 0 to (2-1) columns:
table = pandas.DataFrame(columns=range(0,4), index = range(0, nrows))

In [0]:
# Fill the dataframe with data:
for row in range(nrows):
  # This is going to take some time... Uncomment to see the "progress bar" :D
  #print(url)
  resp = requests.get(urls[row])
  bs = BeautifulSoup(resp.text)

  date = bs.find('p', text = re.compile('DEALING DATE')).text
  table.iat[row,0] = date.split()[2]

  nav = bs.find('p', text = re.compile('NAV PER SHARE')).text
  table.iat[row,1] = float(nav.split()[4])

  shares = bs.find('p', text = re.compile('NUMBER OF SHARES')).text
  table.iat[row,2] = int(shares.split()[5])

  table.iat[row,3] = fnav*ishares

In [0]:
# Rename the columns:
table.columns = ['Date', 'NAV per share', 'Number of shares', 'Total NAV']
table

In [0]:
# Set the date column to datetime type:
table['Date'] = pandas.to_datetime(table['Date'], dayfirst=True)
table

In [0]:
# Do some type conversions:
table['NAV per share'] = pandas.to_numeric(table['NAV per share'])
table['Number of shares'] = pandas.to_numeric(table['Number of shares'])
table['Total NAV'] = pandas.to_numeric(table['Total NAV'])
# Change the index column:
table.index = pandas.DatetimeIndex(table['Date'])
table

In [0]:
# Get rid of the data column since it copies the index column now:
table = table.drop(columns=['Date']).dropna()
table

In [0]:
# Inverse the dataframe:
table = table.iloc[::-1]
table